# 4xCO2 curvature

Test the claim that the models in which the performance of the RFMIP fixed-SST and implied ERF differ the most are the ones with the greatest Gregory plot curvature.

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as pl
import glob
from scipy.io.idl import readsav
from scipy.stats import linregress

## Set up plotting defaults

In [ ]:
pl.rcParams['font.size'] = 9
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.top'] = True

In [ ]:
colors = {
    'cmip5': '#cc2323',
    'cmip6': '#2551cc',
}

## Grab preprocessed data

In [ ]:
with open('../data_output/branch_points.json', 'r') as f:
    branch_points = json.load(f)

In [ ]:
models = sorted(list(branch_points['abrupt-4xCO2'].keys()))
models

In [ ]:
piControls = {
    'CanESM5': 'r1i1p1f1',
    'CNRM-CM6-1': 'r1i1p1f2',
    'GFDL-CM4': 'r1i1p1f1',
    'GISS-E2-1-G': 'r1i1p1f1',
    'HadGEM3-GC31-LL': 'r1i1p1f1',
    'IPSL-CM6A-LR': 'r1i1p1f1',
    'MIROC6': 'r1i1p1f1',
    'NorESM2-LM': 'r1i1p1f1',
}

In [ ]:
piControl = {}
experiments = {}

In [ ]:
for model in models:
    piControl[model] = pd.read_csv('../data_output/cmip6/%s/%s/piControl.csv' % (model, piControls[model]))

In [ ]:
delta_N = {}
delta_T = {}

for experiment in ['abrupt-4xCO2']:
    nyears = 150
    delta_N[experiment] = {}
    delta_T[experiment] = {}
    for model in branch_points[experiment].keys():
        delta_N[experiment][model] = {}
        delta_T[experiment][model] = {}
        for run in list(branch_points[experiment][model].keys()):
            data = pd.read_csv('../data_output/cmip6/%s/%s/%s.csv' % (model, run, experiment))
            index_start = branch_points[experiment][model][run]
            N_4x = data['rsdt'].values[:nyears] - data['rsut'].values[:nyears] - data['rlut'].values[:nyears]
            piC = piControl[model]
            N_piControl = (
                piC['rsdt'][index_start:index_start+nyears].values - 
                piC['rsut'][index_start:index_start+nyears].values - 
                piC['rlut'][index_start:index_start+nyears].values
            )
            delta_N[experiment][model] = N_4x - N_piControl
            delta_T[experiment][model] = data['tas'].values[:nyears] - piC['tas'][index_start:index_start+nyears].values

## Plot fig. S2 and get data for table S2

In [ ]:
# Hand order models by slope
models = ['GISS-E2-1-G', 'MIROC6', 'CNRM-CM6-1', 'CanESM5', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'GFDL-CM4', 'NorESM2-LM']

experiment='abrupt-4xCO2'
fig, ax = pl.subplots(4, 2, figsize=(19/2.54, 23/2.54))
for i, model in enumerate(models):
    col = i%2
    row = i//2
    ax[row,col].scatter(delta_T[experiment][model], delta_N[experiment][model], color=colors['cmip6'])
    ax[row,col].set_title('(%s) %s' % (chr(97+i), model))
    ax[row,col].set_xlabel('$\Delta T$ (K)')
    ax[row,col].set_ylabel('$\Delta N$ (W m$^{-2}$)')
    ax[row,col].grid()

    regress001_020 = linregress(delta_T[experiment][model][:20], delta_N[experiment][model][:20])
    regress021_150 = linregress(delta_T[experiment][model][20:], delta_N[experiment][model][20:])
    print(model, regress001_020.slope, regress021_150.slope, regress001_020.slope/regress021_150.slope)
    x_min001_020 = np.min(delta_T[experiment][model][:20])
    x_max001_020 = np.max(delta_T[experiment][model][:20])
    y_min001_020 = np.min(delta_N[experiment][model][:20])
    y_max001_020 = np.max(delta_N[experiment][model][:20])
    x_min021_150 = np.min(delta_T[experiment][model][20:])
    x_max021_150 = np.max(delta_T[experiment][model][20:])
    y_min021_150 = np.min(delta_N[experiment][model][20:])
    y_max021_150 = np.max(delta_N[experiment][model][20:])
    
    # plot regression slope first 20
    ax[row,col].plot(np.linspace(x_min001_020-0.2, x_max001_020+0.2), regress001_020.slope*np.linspace(x_min001_020-0.2, x_max001_020+0.2)+regress001_020.intercept, color='k', label='Years 1-20')

    # plot regression slope 21-150
    ax[row,col].plot(np.linspace(x_min021_150-0.2, x_max021_150+0.2), regress021_150.slope*np.linspace(x_min021_150-0.2, x_max021_150+0.2)+regress021_150.intercept, color='k', ls='--', label='Years 21-150')

    
#    # plot 1:1 line
#    ax[row,col].plot(np.linspace(x_min-0.2, x_max+0.2), np.linspace(x_min-0.2, x_max+0.2), color='0.4', ls='--', label='1:1 line')
    
    # axis bounds
    ax[row,col].set_xlim(0, 9)
    ax[row,col].set_ylim(0, 8)
    
    #ax[row,col].legend()
    if model not in ['GFDL-CM4', 'IPSL-CM6A-LR', 'MIROC6', 'NorESM2-LM', 'GISS-E2-1-G']:
        ax[row,col].text(0.02,0.16,r'$\lambda_{1\rightarrow 20} = %4.2f$ W m$^{-2}$ K$^{-1}$' % regress001_020.slope, ha='left', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
        ax[row,col].text(0.02,0.02,r'$\lambda_{21\rightarrow 150} = %4.2f$ W m$^{-2}$ K$^{-1}$' % regress021_150.slope, ha='left', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
        ax[row,col].text(0.02,0.32,r'curvature = %4.2f' % (regress001_020.slope/regress021_150.slope), ha='left', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
    else:
        ax[row,col].text(0.98,0.83,r'$\lambda_{1\rightarrow 20} = %4.2f$ W m$^{-2}$ K$^{-1}$' % regress001_020.slope, ha='right', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
        ax[row,col].text(0.98,0.69,r'$\lambda_{21\rightarrow 150} = %4.2f$ W m$^{-2}$ K$^{-1}$' % regress021_150.slope, ha='right', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
        ax[row,col].text(0.98,0.57,r'curvature = %4.2f' % (regress001_020.slope/regress021_150.slope), ha='right', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))

fig.tight_layout(rect=[0.02, 0, 1, 1])
pl.savefig('../plots/figS2.pdf')